In [1]:
import pandas as pd
import numpy as np
import time
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import os


In [2]:
columns = ['Destination_Port', 'Flow_Duration', 'Total_Fwd_Packets',
           'Total_Backward_Packets', 'Total_Length_of_Fwd_Packets',
           'Total_Length_of_Bwd_Packets', 'Fwd_Packet_Length_Max',
           'Fwd_Packet_Length_Min', 'Fwd_Packet_Length_Mean',
           'Fwd_Packet_Length_Std', 'Bwd_Packet_Length_Max',
           'Bwd_Packet_Length_Min', 'Bwd_Packet_Length_Mean',
           'Bwd_Packet_Length_Std', 'Flow_Bytes/s', 'Flow_Packets/s',
           'Flow_IAT_Mean', 'Flow_IAT_Std', 'Flow_IAT_Max', 'Flow_IAT_Min',
           'Fwd_IAT_Total', 'Fwd_IAT_Mean', 'Fwd_IAT_Std', 'Fwd_IAT_Max',
           'Fwd_IAT_Min', 'Bwd_IAT_Total', 'Bwd_IAT_Mean', 'Bwd_IAT_Std',
           'Bwd_IAT_Max', 'Bwd_IAT_Min', 'Fwd_PSH_Flags', 'Bwd_PSH_Flags',
           'Fwd_URG_Flags', 'Bwd_URG_Flags', 'Fwd_Header_Length',
           'Bwd_Header_Length', 'Fwd_Packets/s', 'Bwd_Packets/s',
           'Min_Packet_Length', 'Max_Packet_Length', 'Packet_Length_Mean',
           'Packet _Length_Std', ' Packet_Length_Variance', 'FIN_Flag_Count',
           'SYN_Flag_Count', 'RST_Flag_Count', 'PSH_Flag_Count',
           'ACK_Flag_Count', 'URG_Flag_Count', 'CWE_Flag_Count',
           'ECE_Flag_Count', 'Down/Up_Ratio', 'Average_Packet_Size',
           'Avg_Fwd_Segment_Size', 'Avg_Bwd_Segment_Size',
           'Fwd_Header_Length.1', 'Fwd_Avg_Bytes/Bulk', '_Fwd_Avg_Packets/Bulk',
           'Fwd_Avg_Bulk_Rate', 'Bwd_Avg_Bytes/Bulk', 'Bwd_Avg_Packets/Bulk',
           'Bwd_Avg_Bulk_Rate', 'Subflow_Fwd_Packets', 'Subflow_Fwd_Bytes',
           'Subflow_Bwd_Packets', 'Subflow_Bwd_Bytes', 'Init_Win_bytes_forward',
           'Init_Win_bytes_backward', 'act_data_pkt_fwd',
           'min_seg_size_forward', 'Active_Mean', 'Active_Std', 'Active_Max',
           'Active Min', 'Idle_Mean', 'Idle_Std', 'Idle_Max', 'Idle_Min',
           'Label']

csvfile = "/root/data/fxg/csv"
all_scv_list = os.listdir(csvfile)
for single_csv in all_scv_list:
    single_data = pd.read_csv(os.path.join(csvfile, single_csv), header=1)
    if single_csv == all_scv_list[0]:
        data = single_data
    else:
        data = pd.concat([data, single_data], ignore_index=True)
data.columns = columns
data.replace([np.inf, -np.inf], np.nan, inplace=True)
data.dropna(inplace=True)
data.drop_duplicates(inplace=True)
data.Label[data['Label'] != 'BENIGN'] = 1
data.Label[data['Label'] == 'BENIGN'] = 0

/tmp/ipykernel_15176/1641415974.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.Label[data['Label'] != 'BENIGN'] = 1
/tmp/ipykernel_15176/1641415974.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.Label[data['Label'] == 'BENIGN'] = 0


In [3]:
# #硬编码
# encode=LabelEncoder()
# encode.fit(col)
# data['Label']=encode.transform(data['Label'])


In [4]:
X = data.drop(['Label'], axis=1)
labelencoder_x=LabelEncoder()
for i in columns[:-1]:
      X[i]=labelencoder_x.fit_transform(X[i])
y = data['Label'].astype(int)
del data
# 标准化
transfer = MinMaxScaler(feature_range=(2, 3))
X = transfer.fit_transform(X)

# 降维
transfer = VarianceThreshold(threshold=0.0)
X = transfer.fit_transform(X)
# 主成分分析
transfer = PCA(n_components=0.9)
X = transfer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=20)
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


In [5]:
# data[data.isnull().values==True]
# data
# transfer = MinMaxScaler(feature_range=(2, 3))
# data = transfer.fit_transform(data[data.keys()])


In [6]:
def KNN(train_X, train_Y, test_X, test_Y):
    print('[KNN] train...')
    t1 = time.time()
    knn = KNeighborsClassifier(n_neighbors=5)
    model = knn.fit(train_X, train_Y)
    y_hat = model.predict(test_X)
    acc = accuracy_score(test_Y, y_hat)
    t2 = time.time()
    print('acc:', acc)
    print('using time:', t2 - t1, 'sec')
    matrix = confusion_matrix(test_Y, y_hat)
    print(matrix)
    report = classification_report(test_Y, y_hat)
    print(report)
    error = mean_squared_error(test_Y, y_hat)
    print("KNN-均方差为：\n", error)
    print('-' * 20)
    # plot_confusion_matrix(matrix, label, True, 'KNN Confusion matrix')
KNN(X_train,y_train,X_test,y_test)

[KNN] train...
acc: 0.9994721240914444
using time: 4.164442777633667 sec
[[48587     8]
 [   18   641]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     48595
           1       0.99      0.97      0.98       659

    accuracy                           1.00     49254
   macro avg       0.99      0.99      0.99     49254
weighted avg       1.00      1.00      1.00     49254

KNN-均方差为：
 0.0005278759085556503
--------------------


In [ ]:
# estimator = KNeighborsClassifier(n_neighbors=3)
# estimator.fit(X_train, y_train.astype(int))
# classification_report(y_test, estimator.predict(X_test))
# print("KNN classification_report==\n", classification_report(
#     y_test, estimator.predict(X_test)))
# error = mean_squared_error(y_test, estimator.predict(X_test))
# print("KNN-均方差为：\n", error)


In [ ]:
# from sklearn.neural_network import MLPClassifier
# print("Neural Network")
# mlpc = MLPClassifier(hidden_layer_sizes=(11, 11, 11), max_iter=500)
# mlpc.fit(X_train, y_train)
# pred_mlpc = mlpc.predict(X_test)
# print("Neural Network classification_report==\n",
#       classification_report(y_test, pred_mlpc))
# print("Neural Network confusion_matrix==\n",
#       confusion_matrix(y_test, pred_mlpc))
# error = mean_squared_error(y_test, pred_mlpc)
# print("神经网络-均方误差为：\n", error)


In [ ]:
#随机深林
import time
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
def RF(train_X, train_Y, test_X, test_Y):
    print("Random Forest Classifier")
    t1 = time.time()
    rfc = RandomForestClassifier()
    rfc.fit(train_X, train_Y)
    Y_pred = rfc.predict(test_X)
    acc = accuracy_score(test_Y, Y_pred)
    t2 = time.time()
    print('acc:', acc)
    print('using time:', t2 - t1, 'sec')
    matrix = confusion_matrix(test_Y, Y_pred)    
    print(matrix)
    error = mean_squared_error(y_test, pred_rfc)
    print("随机森林-均方差为：\n", error)
    report = classification_report(test_Y, Y_pred)
    print(report)
    print('-' * 40)
    # plot_confusion_matrix(matrix, label, True, 'RF Confusion matrix')
RF(X_train,y_train,X_test,y_test)

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# print("Random Forest Classifier")
# # n_estimators，表示选择多少棵树来构造随机森林；具体解释看《边学边练超系统掌握人工智机器学习算法 传智播客》
# rfc = RandomForestClassifier(n_estimators=200)
# rfc.fit(X_train, y_train)
# pred_rfc = rfc.predict(X_test)
# print("Random Forest classification_report==\n",
#       classification_report(y_test, pred_rfc))
# print("Random Forest confusion_matrix==\n",
#       confusion_matrix(y_test, pred_rfc))
# error = mean_squared_error(y_test, pred_rfc)
# print("随机森林-均方差为：\n", error)


In [ ]:
from sklearn.svm import SVC
def SVM(train_X, train_Y, test_X, test_Y):
    print('[SVM] train ...')
    # train_Y = [np.where(r == 1)[0][0] for r in train_Y]
    # test_Y = [np.where(r == 1)[0][0] for r in test_Y]
    t1 = time.time()
    clf = SVC(decision_function_shape='ovr', max_iter=300, kernel='rbf')
    model = clf.fit(train_X, train_Y)
    y_hat = model.predict(test_X)
    acc = accuracy_score(test_Y, y_hat)
    t2 = time.time()
    print('acc:', acc)
    print('using time:', t2 - t1, 'sec')
    matrix = confusion_matrix(test_Y, y_hat)
    print(matrix)
    report = classification_report(test_Y, y_hat)
    print(report)
    error = mean_squared_error(y_test, y_hat)
    print("SVM-均方误差为：\n", error)
    print('-' * 20)
    # plot_confusion_matrix(matrix, label, True, 'SVM Confusion matrix')
SVM(X_train,y_train,X_test,y_test)

In [ ]:
# from sklearn.svm import SVC
# from sklearn import svm, metrics
# print("SVM Classifier")
# svmc = svm.SVC()
# svmc.fit(X_train, y_train)
# pred_svmc = svmc.predict(X_test)
# print("SVM classification_report==\n",
#       classification_report(y_test, pred_svmc))
# print("SVM classification_confusion_matrix==\n",
#       confusion_matrix(y_test, pred_svmc))
# error = mean_squared_error(y_test, pred_svmc)
# print("SVM-均方误差为：\n", error)


In [ ]:
from sklearn.naive_bayes import BernoulliNB
def NaiveBayes(train_X, train_Y, test_X, test_Y):
    print('[Naive Bayes] train ...')
    # train_Y = [np.where(r == 1)[0][0] for r in train_Y]
    # test_Y = [np.where(r == 1)[0][0] for r in test_Y]
    t1 = time.time()
    clf = BernoulliNB()
    model = clf.fit(train_X, train_Y)
    y_hat = model.predict(test_X)
    acc = accuracy_score(test_Y, y_hat)
    t2 = time.time()
    print('acc:', acc)
    print('using time:', t2 - t1, 'sec')
    matrix = confusion_matrix(test_Y, y_hat)
    print(matrix)
    report = classification_report(test_Y, y_hat)
    print(report)
    print('-' * 20)
    # plot_confusion_matrix(matrix, label, True, 'NB Confusion matrix')
NaiveBayes(X_train,y_train,X_test,y_test)

In [48]:
#神经网络
from sklearn.neural_network import MLPClassifier
def MLP(train_X, train_Y, test_X, test_Y):
    print('[MLP] train ...')
    t1 = time.time()
    model = MLPClassifier(hidden_layer_sizes=(100,),
                          activation='logistic',
                          solver='adam',
                          learning_rate_init=0.0001,
                          max_iter=2000)
    model.fit(train_X, train_Y)
    y_hat = model.predict(test_X)
    acc = accuracy_score(test_Y, y_hat)
    t2 = time.time()
    print('acc:', acc)
    print('using time:', t2 - t1, 'sec')
    matrix = confusion_matrix(test_Y, y_hat)
    print(matrix)
    report = classification_report(test_Y, y_hat)
    print(report)
    print('-' * 20)
    # plot_confusion_matrix(matrix, label, True, 'MLP Confusion matrix')
MLP(X_train,y_train,X_test,y_test)

[MLP] train ...
acc: 0.997340317537662
using time: 145.4353985786438 sec
[[48587     8]
 [  123   536]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     48595
           1       0.99      0.81      0.89       659

    accuracy                           1.00     49254
   macro avg       0.99      0.91      0.94     49254
weighted avg       1.00      1.00      1.00     49254

--------------------


In [50]:
from sklearn.tree import DecisionTreeClassifier
def DT(train_X, train_Y, test_X, test_Y):
    t1 = time.time()
    clf = DecisionTreeClassifier(max_depth=6)
    model = clf.fit(train_X, train_Y)
    y_hat = model.predict(test_X)
    acc = accuracy_score(test_Y, y_hat)
    t2 = time.time()
    print('acc:', acc)
    print('using time:', t2 - t1, 'sec')
    matrix = confusion_matrix(test_Y, y_hat)
    print(matrix)
    report = classification_report(test_Y, y_hat)
    print(report)
    print('-' * 20)
    # plot_confusion_matrix(matrix, label, True, 'DT Confusion matrix')
DT(X_train,y_train,X_test,y_test)

acc: 0.9991675802980469
using time: 1.0644662380218506 sec
[[48589     6]
 [   35   624]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     48595
           1       0.99      0.95      0.97       659

    accuracy                           1.00     49254
   macro avg       0.99      0.97      0.98     49254
weighted avg       1.00      1.00      1.00     49254

--------------------
